# Homework 1

In [18]:
## 1.1
Using Introspection as a method of modeling Artificial Intelligence processes could be useful, but there are probably better methods. Introspection use as a whole in the realm of Psychology has been in decline in recent history because of the rise of functionalism and behaviorism. Introspection is all about thinking about the way we think, and documenting that thinking process. Perhaps this could be used to program some decision making processes that are meant to mimic human flawed logic, but that is a niche use. The problem is that humans generally have little ability to truly know what they are thinking and more importantly why they are thinking certain things and most importantly humans are quite bad at explaining this in a way a computer could understand. I believe statistical methods like Bayes Theorem and other statistical models are a much better model for Artificial Intelligence processes.


SyntaxError: invalid syntax (<ipython-input-18-93b3d6a4886b>, line 3)

## 1.2

The State of this problem would be an ordered list of all of the cities with the starting location also being the ending location, but that is the only city that appears twice. 
 
An Action would be switching any two cities in the list between the bookends of the start and finish location. This switch will either increase, decrease, or maintain the current length the Salesman must traverse to visit all of the cities and return home.
 
The Local Search Algorithm will attempt to optimize for the total distance accumulated by visiting all cities in the current order of the state.



In [ ]:
import sys
import random
import time
sys.path.insert(0, '../tools/aima')
from search import Problem, hill_climbing, simulated_annealing, exp_schedule, genetic_search


class Town:

    def __init__(self, id):
        self.neighbors = {}
        self.id = id
    
    def setNeighbor(self, id, distance):
        self.neighbors[id] = distance
    
    def getDistance(self, id):
        if(id == self.id):
            return 0
        return self.neighbors[id]

    def __str__(self):
        string = ''
        for key, value in self.neighbors.items():
            string += "Neighbor: " + str(key) + " Distance: " + str(value) + "\n"
        return "Town " + str(self.id) + "\n" + string

class World:

    def __init__(self, numberOfTowns):
        self.towns = {}
        for i in range(numberOfTowns):
            town = Town(i)
            self.towns[i] = town
        for i in range(numberOfTowns):
            for j in range(numberOfTowns):
                if(i == j):
                    continue
                dist = random.randint(1, 100)
                self.setDistance(i, j, dist)
        
        for key, value in self.towns.items():
            print("\n")
            print(value)
            print("\n")


    def setDistance(self, town1, town2, distance):
        self.towns[town1].setNeighbor(town2, distance)
        self.towns[town2].setNeighbor(town1, distance)

    def calculateDistance(self, town1, town2):
        return self.towns[town1].getDistance(town2)


class TSP(Problem):
    """
    State: x value for the sine function variant f(x)
    Move: a new x value delta steps from the current x (in both directions) 
    """
    
    def __init__(self, world, initial, maximum=30, delta=0.001):
        self.world = world
        self.delta = delta
        self.initial = initial
        
        
    def actions(self, state):
        switches = []
        currentPath = state[:]
        for x in currentPath:
            if(x == currentPath[0]):
                continue
            for y in currentPath:
                if(x == currentPath[0]):
                    continue
                switches.append(self.switch(currentPath, x, y))

        return switches
    
    def result(self, stateIgnored, x):
        return x
    
    def value(self, path):
        totalDistance = 0
        # print(path)
        for town in path:
            nextTown = (path.index(town) + 1)
            if(nextTown >= len(path)):
                break
            nextStopDistance = self.world.calculateDistance(town, path[nextTown])
            #print("Current Town is:", town)
            #print("Next Town is:", path[nextTown])
            #print("Distance from", town, "to", path[nextTown], "is", nextStopDistance)
            totalDistance += nextStopDistance
        
        returnHomeDistance = self.world.calculateDistance(path[0], path[-1])
        #print("Return Home:", returnHomeDistance)
        totalDistance += returnHomeDistance

        return -totalDistance

    def switch(self, path, index1, index2):
        newPath = []
        for i in path:
            if(path.index(i) == index1):
                newPath.append(path[index2])
            elif(path.index(i) == index2):
                newPath.append(path[index1])
            else:
                newPath.append(i)
        return newPath

if __name__ == "__main__":
    towns = 10
    world = World(towns)
    path = []
    for i in range(towns):
        path.append(i)
    
    random.shuffle(path)
    print(path)
    tsp = TSP(world, path)
    print('Initial                      x: ' + str(tsp.initial)
          + '\t\tvalue: ' + str(-tsp.value(path))
          )

    startHill = time.time()
    # Solve the problem using hill-climbing.
    hill_solution = hill_climbing(tsp)
    endHill = time.time()
    print('Hill-climbing solution       x: ' + str(hill_solution)
          + '\tvalue: ' + str(-tsp.value(hill_solution))
          )
    print("Time taken was", round(endHill - startHill, 7))

    startAnneal = time.time()
    annealing_solution = simulated_annealing(
        tsp,
        exp_schedule(k=20, lam=0.005, limit=1000)
    )
    endAnneal = time.time()
    print('Simulated annealing solution x: ' + str(annealing_solution)
          + '\tvalue: ' + str(-tsp.value(annealing_solution))
          )
    print("Time taken was", round(endAnneal - startAnneal, 7))    

The Simulated Annealing solution seems to work better overall. With 5 towns or less, there is no difference as both find the solution, but since Simulated Annealing has the potential to jump across troughs that seperate local maxima, it is better at optimizing a shorter route. To fit the problem into the algorithm's frame work, the value function returns the distance as a negative integer so that the algorithm optimizes correctly. The Hill climbing algorithm gets stuck at local maximums too easily and there are many local maximums, so the chances of it finding a relatively high local maximum is pretty low as the number of towns grows.

An interesting problem. Definitely many different ways about creating this Problem object for the algorithms to solve.

In [13]:
## 1.3

{'cs108': 'norman nh064 mwf1130', 'cs112': 'adams sb354 mwf1130', 'cs214': 'adams sb354 tth1030', 'stat343': 'pruim nh064 tth130', 'cs336': 'vanderlinden nh064 tth1030', 'cs300': 'schuurman sb354 tth130'}


In [ ]:
import sys
sys.path.insert(0, '../tools/aima')
from csp import CSP, parse_neighbors, min_conflicts


def Scheduler():
    """ Returns an instance of the CSP class. """
    courses = "cs108 cs112 cs214 stat343 cs336 cs300".split()
    profs = "norman adams schuurman pruim vanderlinden".split()
    slots = "mwf900 mwf1130 tth1030 tth130".split()
    rooms = "sb354 nh064".split()
        
    variables = courses
    assignments = {}
    assignments['cs108'] = "norman"
    assignments['cs112'] = "adams"
    assignments['cs214'] = "adams"
    assignments['stat343'] = "pruim"
    assignments['cs336'] = "vanderlinden"
    assignments['cs300'] = "schuurman"
    neighbors = parse_neighbors("""
    cs108: norman; cs112: adams; 
    cs214: adams; stat343: pruim; 
    cs336: vanderlinden; cs300: schuurman
    """, variables)
    domains = {}
    for course in courses:
        domains[course] = []
    for course in courses:
        for prof in profs:
            for room in rooms:
                for slot in slots:
                    domains[course].append(prof + " " + room + " " + slot)
    
    for type in [courses]:
        for A in type:
            for B in type:
                if A != B:
                    if B not in neighbors[A]:
                        neighbors[A].append(B)
                    if A not in neighbors[B]:
                        neighbors[B].append(A)

    def scheduler_constraints(A, a, B, b, recurse=0):
        ADomain = a.split()
        BDomain = b.split()
        A_Prof = ADomain[0]
        B_Prof = BDomain[0]
        A_Room = ADomain[1]
        B_Room = BDomain[1]
        A_Slot = ADomain[2]
        B_Slot = BDomain[2]
        A_Course = A
        B_Course = B
        
        if(A_Prof == B_Prof and A_Slot == B_Slot):
            return False
        if(A_Room == B_Room and A_Slot == B_Slot):
            return False

        if('norman' in a and A == 'cs108'):
            return True
        if('adams' in a and A == 'cs112'):
            return True
        if('adams' in a and A == 'cs214'):
            return True
        if('pruim' in a and A == 'stat343'):
            return True
        if('vanderlinden' in a and A == 'cs336'):
            return True
        if('schuurman' in a and A == 'cs300'):
            return True
        if(A in courses and B in courses):
            return False
        if(recurse == 0):
            return scheduler_constraints(B, b, A, a, 1)
        return True
    
    return CSP(variables, domains, neighbors, scheduler_constraints)

if __name__ == "__main__":
    schedule = Scheduler()
    solution = min_conflicts(schedule)
    print(solution)



